In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import os
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, MultiHeadAttention, Dropout, Embedding, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential, Model

SEED_TEXTS = ["محبت", "دل", "شام", "یاد", "خوشی"] 
TEMPERATURES = [0.7, 1.0, 1.3]
NEXT_WORDS = 15 

os.makedirs('../results/generated_text', exist_ok=True)

print(" Loading Tokenizer and Data...")
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

data = np.load('../data/processed/ready_data.npz')
max_sequence_len = data['X_train'].shape[1] + 1 
print(f"Tokenizer Loaded. Vocab Size: {len(tokenizer.word_index)}")

# Defining custom layers for Transformer model

@tf.keras.utils.register_keras_serializable()
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim),])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate,
        })
        return config

@tf.keras.utils.register_keras_serializable()
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

#Generation functions

def generate_poetry(model, seed_text, next_words, temperature, max_sequence_len):
    output_text = seed_text # Starting with the seed text we want to generate from
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([output_text])[0] # Convert current text which is a string to sequence of tokens 
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # Pad sequences to the required length
        
        predictions = model.predict(token_list, verbose=0)[0] # Getting model predictions for the next word 
        predictions = np.log(predictions + 1e-7) / temperature # Apply temperature to the logits to control the randomness of the predictions
        exp_preds = np.exp(predictions) # Exponentiate the logits
        predictions = exp_preds / np.sum(exp_preds) # Normalize to get probabilities
        
        predicted_id = np.random.choice(len(predictions), p=predictions) # Sample the next word ID based on probabilities
        
        predicted_word = "" # Finding  the word corresponding to the predicted ID
        for word, index in tokenizer.word_index.items():
            if index == predicted_id:
                predicted_word = word
                break
        
        if predicted_word == "": break
        output_text += " " + predicted_word
        
    return output_text

#  Generation loop

models_to_test = ["RNN", "LSTM", "Transformer"]
optimizers_to_test = ["Adam", "RMSprop", "SGD"]

full_generation_log = []

print(f"Starting Generation for {len(models_to_test) * len(optimizers_to_test)} Models...")

for model_name in models_to_test:
    for opt_name in optimizers_to_test:
        model_filename = f"../models/{model_name}_{opt_name}.keras"
        
        if not os.path.exists(model_filename):
            print(f"⚠️ Model not found: {model_filename}")
            continue
            
        print(f"\n Generating with: {model_name} + {opt_name}")
        
        try:
            model = load_model(model_filename, custom_objects={
                'TokenAndPositionEmbedding': TokenAndPositionEmbedding,
                'TransformerBlock': TransformerBlock
            })
        except Exception as e:
            print(f" Error loading {model_name}: {e}")
            continue

        for seed in SEED_TEXTS:
            for temp in TEMPERATURES:
                try:
                    generated_text = generate_poetry(model, seed, NEXT_WORDS, temp, max_sequence_len)
                    full_generation_log.append({
                        "Model": model_name,
                        "Optimizer": opt_name,
                        "Seed": seed,
                        "Temperature": temp,
                        "Generated Text": generated_text
                    })
                except Exception as e:
                    print(f"Error generating: {e}")

# Saving results
df_results = pd.DataFrame(full_generation_log)
df_results.to_csv('../results/generated_text/all_generations.csv', index=False)

print("\n Generation Complete! Saved to ../results/generated_text/all_generations.csv")
print(df_results[['Model', 'Optimizer', 'Generated Text']].head())

🔵 Loading Tokenizer and Data...
Tokenizer Loaded. Vocab Size: 10224
🚀 Starting Generation for 9 Models...

📝 Generating with: RNN + Adam

📝 Generating with: RNN + RMSprop

📝 Generating with: RNN + SGD

📝 Generating with: LSTM + Adam

📝 Generating with: LSTM + RMSprop

📝 Generating with: LSTM + SGD

📝 Generating with: Transformer + Adam


C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'token_and_position_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:4



📝 Generating with: Transformer + RMSprop


C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'token_and_position_embedding_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.


📝 Generating with: Transformer + SGD


C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'token_and_position_embedding_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Users\sherr\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.


✅ Generation Complete! Saved to ../results/generated_text/all_generations.csv
  Model Optimizer                                     Generated Text
0   RNN      Adam  محبت میں ہے بات ہی ہی سا اسی کو آ ہے کیا وہ یہ...
1   RNN      Adam  محبت ولی یہ جب لفظ اس ہی نہ ہماری دیکھتا کے یا...
2   RNN      Adam  محبت آ قدر سما و آزمائش سرشار سمانے افسون قبلہ...
3   RNN      Adam  دل دیا تو ہے نہیں ہے بس بھی کہ عشق تو اس سے مر...
4   RNN      Adam  دل لیا کاسۂ ہم سے اپنی ظلم نہیں کہ راہ میں مجھ...
